# Volume 3: Discrete Hidden Markov Models
    Benj McMullin
    Math 405
    2/13/2024

In [15]:
import numpy as np
import copy
import string
import codecs
from hmmlearn import hmm as hmmlearn

# Problems 1-5
This is the HMM class that you will be adding functions to throughout the lab.

In [2]:
class HMM(object):
    """
    Finite state space hidden Markov model.
    """
    
    # Problem 1
    def __init__(self, A, B, pi):
        """
        Initialize an HMM with parameters A, B, and pi.
        """
        self.A = A
        self.B = B
        self.pi = pi
    
    def init_matrix(self, z):
        # Initialize alpha
        T = z.shape[0]
        return np.empty((T, self.A.shape[0])), T
    
    # Problem 2
    def forward_pass(self, z):
        """
        Compute the forward probability matrix.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        alpha : ndarray of shape (T, n)
            The forward probability matrix
        """
        # Initialize alpha
        alpha,T = self.init_matrix(z)

        # Set the first row of alpha
        alpha[0] = self.pi * self.B[z[0]]

        # Iterate through the rest of alpha
        for t in range(1, T):
            alpha[t] = self.B[z[t],:] * np.sum(alpha[t-1,:] * self.A, axis=1)

        return alpha
    
    # Problem 4
    def backward_pass(self, z):
        """
        Compute the backward probability matrix and gamma values.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        beta : ndarray of shape (T, n)
            The backward probability matrix
        gamma : ndarray of shape (T, n)
            The state probability matrix
        """
        # Initialize beta
        beta,T = self.init_matrix(z)
        beta[-1] = np.ones(self.A.shape[0])

        # Calcualte beta_t's
        for t in range(T-2, -1, -1):
            beta[t] = (self.B[z[t+1],:] * beta[t+1,:]) @ self.A

        # Calculate gamma using forward pass
        gamma = self.forward_pass(z)*beta
        gamma = gamma / np.sum(gamma, axis=1).reshape(-1,1)

        return beta, gamma
    
    # Problem 5
    def viterbi_algorithm(self, z):
        """
        Compute the most likely hidden state sequence using the Viterbi algorithm.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence

        Returns
        -------
        x*: ndarray of shape (T,)
            The most likely state sequence
        """
        # Initialize eta and x*
        eta,T = self.init_matrix(z)
        x_str = np.empty(T, dtype=int)
        eta[0] = self.pi * self.B[z[0]]

        # Iterate through the rest of eta
        for t in range(1, T):
            eta[t] = self.B[z[t],:] * np.max(eta[t-1,:]*self.A, axis=1)

        # Calculate x* from eta
        x_str[-1] = np.argmax(eta[-1])
        for t in range(T-2, -1, -1):
            x_str[t] = np.argmax(eta[t]*self.A[:,x_str[t+1]])
        
        return x_str

# Problem 2 test case

Use the following HMM and code to test your HMM class.
Compare the output to `forward_pass` with the lab pdf.

In [3]:
pi = np.array([.6, .4])
A = np.array([[.7, .4],[.3, .6]])
B = np.array([[.1,.7],[.4, .2],[.5, .1]])
z_example = np.array([0, 1, 0, 2])
example_hmm = HMM(A, B, pi)

alpha=example_hmm.forward_pass(z_example)
print(np.sum(alpha[-1,:]))

0.009629599999999997


# Problem 3

Consider the following (very simplified) model of the price of a stock over time as an HMM.
The observation states will be the change in the value of the stock.
For simplicity, we will group these into five values: large decrease, small decrease, no change, small increase, large increase, labeled as integers from 0 to 4.
The hidden state will be the overall trends of the market.
We'll consider the market to have three possible states: declining in value (bear market), not changing in value (stagnant), and increasing in value (bull market), labeled as integers from 0 to 2.
Let the HMM modeling this scenario have parameters
$$
\boldsymbol\pi=\begin{bmatrix}
1/3 \\ 1/3 \\ 1/3
\end{bmatrix},
\quad
A=\begin{bmatrix}
0.5 & 0.3 & 0 \\
0.5 & 0.3 & 0.3 \\
0 & 0.4 & 0.7
\end{bmatrix},
\quad
B=\begin{bmatrix}
0.3 & 0.1 & 0 \\
0.3 & 0.2 & 0.1 \\
0.3 & 0.4 & 0.3 \\
0.1 & 0.2 & 0.4 \\
0 & 0.1 & 0.2
\end{bmatrix}
$$
The file `stocks.npy` contains a sequence of 50 observations drawn from this HMM.
What is the probability of this observation sequence given these model parameters?
Use your implementation of the forward pass algorithm from Problem 2 to find the answer.
Note that the answer is very small, because there are lots of possible observation sequences.

In [4]:
# HMM parameter setup
pi = np.array([1/3, 1/3, 1/3])
A = np.array([
    [0.5, 0.3, 0.0],
    [0.5, 0.3, 0.3],
    [0.0, 0.4, 0.7]
])
B = np.array([
    [0.3, 0.1, 0.0],
    [0.3, 0.2, 0.1],
    [0.3, 0.4, 0.3],
    [0.1, 0.2, 0.4],
    [0.0, 0.1, 0.2]
])

In [7]:
# Load stocks data
data = np.load('stocks.npy')

# Define HMM
hmm = HMM(A, B, pi)

# Run the forward pass
alpha = hmm.forward_pass(data)
print(f"Probability: {np.sum(alpha[-1, :])}")

Probability: 6.671115114537777e-34


# Problem 4

Create a method `backward_pass` in your HMM class to implement the backward pass algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return two arrays of the $\beta_t(i)$ and $\gamma_t(i)$ values.

Test your function on the example HMM, and compare the output with the lab pdf.

With your function and the stock model from Problem 3, answer the following question: given the observation sequence in `stocks.npy`, what is the most likely initial hidden state $X_0$?

In [8]:
# Test case; compare your output with what is in the lab pdf
beta, gamma = example_hmm.backward_pass(z_example)
print(beta)
print(gamma)

[[0.0302  0.02792]
 [0.0812  0.1244 ]
 [0.38    0.26   ]
 [1.      1.     ]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [9]:
# Load stocks data
data = np.load('stocks.npy')

# Define HMM
hmm = HMM(A, B, pi)

# Run the backward pass
beta, gamma = hmm.backward_pass(data)
print("Most likely initial state:", np.argmax(beta[0]))

Most likely initial state: 0


# Problem 5
Creating a method `viterbi_algorithm` in your HMM class to implement the Viterbi algorithm.
This function should accept the observation sequence $\mathbf{z}$ and return the most likely state sequence $\mathbf{x}^*$.

Test your function on the example HMM and compare output with the lab pdf.

Apply your function to the stock market HMM from Problem 3.
With the observaition sequence from `stocks.npy`, what is the most likely sequence of hidden states?
Is the initial state of the most likely sequence the same as the most likely initial state you found in Problem 4?

In [10]:
# Test case
xstar = example_hmm.viterbi_algorithm(z_example)
print(xstar)

[1 1 1 0]


In [11]:
# Run viterbi
xstar = hmm.viterbi_algorithm(data)
print(xstar)

[2 2 1 0 0 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 0 0 0 1 2 2 2 2 2
 2 2 2 2 2 2 2 1 2 2 1 0 1]


*Write your observations here.*

# Problem 6

Train a `hmmlearn.hmm.CategoricalHMM` on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values (26 lower case characters and 1 whitespace character).
Use `n_iter=200` and `tol=1e-4`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [12]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    """
    Reads in the file and prepares it for use in an HMM.
    Returns:
        symbols (dict): a dictionary that maps characters to their integer values
        obs_sequence (ndarray): an array of integers representing the read-in text
    """
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case
    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)
    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))
    # convert the data to a NumPy array of symbols
    a = np.array(list(data))
    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}
    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)
    return symbols, obs_sequence

In [16]:
symbols, obs = prep_data('declaration.txt')

# Define HMM
hmm = hmmlearn.CategoricalHMM(n_components=3, n_iter=200, tol=1e-4)
hmm.fit(obs.reshape(-1, 1), lengths=[obs.shape[0]])

# Print out the HMM parameters
B = hmm.emissionprob_.T
print('The first state represents vowels and the second state represents consonants')
for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))


The first state represents vowels and the second state represents consonants
 , 0.2713, 0.0687
a, 0.1344, 0.0000
b, 0.0000, 0.0352
c, 0.0000, 0.0502
d, 0.0009, 0.0188
e, 0.2393, 0.0000
f, 0.0000, 0.0549
g, 0.0009, 0.0110
h, 0.0059, 0.0000
i, 0.1266, 0.0000
j, 0.0000, 0.0000
k, 0.0009, 0.0000
l, 0.0000, 0.0392
m, 0.0012, 0.0339
n, 0.0000, 0.1872
o, 0.1374, 0.0000
p, 0.0006, 0.0371
q, 0.0000, 0.0000
r, 0.0000, 0.1158
s, 0.0144, 0.1147
t, 0.0000, 0.1753
u, 0.0589, 0.0000
v, 0.0000, 0.0180
w, 0.0000, 0.0349
x, 0.0000, 0.0035
y, 0.0074, 0.0000
z, 0.0000, 0.0016


# Problem 7

Repeat the same calculation with `WarAndPeace.txt` with 2 hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [17]:
symbols, obs = prep_data('WarAndPeace.txt')

# Define HMM
hmm = hmmlearn.CategoricalHMM(n_components=3, n_iter=200, tol=1e-4)
hmm.fit(obs.reshape(-1, 1), lengths=[obs.shape[0]])

# Print out the HMM parameters
B = hmm.emissionprob_.T
for i in range(len(B)):
    print(u"{}, {:0.4f}, {:0.4f}".format(symbols[i], *B[i,:]))


 , 0.4188, 0.0342
а, 0.0000, 0.1958
б, 0.0122, 0.0000
в, 0.0494, 0.0000
г, 0.0174, 0.0000
д, 0.0262, 0.0000
е, 0.0361, 0.1556
ж, 0.0061, 0.0000
з, 0.0205, 0.0000
и, 0.0000, 0.1490
й, 0.0271, 0.0000
к, 0.0348, 0.0000
л, 0.0376, 0.0000
м, 0.0275, 0.0000
н, 0.0401, 0.0000
о, 0.0000, 0.2678
п, 0.0244, 0.0000
р, 0.0165, 0.0000
с, 0.0929, 0.0000
т, 0.0355, 0.0000
у, 0.0007, 0.0651
ф, 0.0017, 0.0000
х, 0.0086, 0.0000
ц, 0.0009, 0.0000
ч, 0.0164, 0.0000
ш, 0.0047, 0.0000
щ, 0.0003, 0.0000
ъ, 0.0000, 0.0000
ы, 0.0000, 0.0418
ь, 0.0000, 0.0497
э, 0.0079, 0.0000
ю, 0.0136, 0.0034
я, 0.0223, 0.0374
ё, 0.0000, 0.0001


Vowels are а, е, и, о, с, у, ы, ь, ю, я